https://bit.ly/DSNN-1-intro

# Классификация мошенечества с кредитными картами

## Introduction

This example looks at the
[Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/)
dataset to demonstrate how
to train a classification model on data with highly imbalanced classes.

## First, vectorize the CSV data

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:
features = data.drop(columns=['Class']).values
targets = data[['Class']].values.astype('uint8')

In [4]:
features.shape

(284807, 30)

In [5]:
targets.shape

(284807, 1)

In [6]:
targets

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [7]:
features

array([[ 0.00000000e+00, -1.35980713e+00, -7.27811733e-02, ...,
         1.33558377e-01, -2.10530535e-02,  1.49620000e+02],
       [ 0.00000000e+00,  1.19185711e+00,  2.66150712e-01, ...,
        -8.98309914e-03,  1.47241692e-02,  2.69000000e+00],
       [ 1.00000000e+00, -1.35835406e+00, -1.34016307e+00, ...,
        -5.53527940e-02, -5.97518406e-02,  3.78660000e+02],
       ...,
       [ 1.72788000e+05,  1.91956501e+00, -3.01253846e-01, ...,
         4.45477214e-03, -2.65608286e-02,  6.78800000e+01],
       [ 1.72788000e+05, -2.40440050e-01,  5.30482513e-01, ...,
         1.08820735e-01,  1.04532821e-01,  1.00000000e+01],
       [ 1.72792000e+05, -5.33412522e-01, -1.89733337e-01, ...,
        -2.41530880e-03,  1.36489143e-02,  2.17000000e+02]])

## Prepare a validation set

In [8]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analyze class imbalance in the targets

In [9]:
counts = np.bincount(train_targets[:, 0])
counts

array([227429,    417], dtype=int64)

In [10]:
print(
    f"Number of positive samples in training data: {counts[1]} ({100 * float(counts[1]) / len(train_targets):.2f}% of total)"
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


## Normalize the data using training set statistics

In [11]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean

std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Build a binary classification model

### Dropout

Метод регуляризации искусственных нейронных сетей, предназначен для уменьшения переобучения сети за счет предотвращения сложных адаптаций отдельных нейронов на тренировочных данных во время обучения.

Характеризует исключение определённого процента (например 50%) случайных нейронов на разных итерациях во время обучения нейронной сети. В результате  обучение происходит более общее, нет надежды на определенные нейроны. Такой приём значительно увеличивает скорость обучения, качество обучения на тренировочных данных, а также повышает качество предсказаний модели на новых тестовых данных.

На моменте предсказания все нейроны включаются обратно, dropout не используется.

<img src='https://drive.google.com/uc?export=view&id=1KQrdTDanDkLhf8Kn8c5ryjqN2acwYWII' width=500>

In [12]:
from tensorflow import keras

hid_size = 256
model = keras.Sequential(
    [
        keras.layers.Dense(hid_size, activation="relu", input_shape=(train_features.shape[-1],)),  
        keras.layers.Dense(hid_size, activation="relu"),         
        keras.layers.Dropout(0.3),        
        keras.layers.Dense(hid_size, activation="relu"),         
        keras.layers.Dropout(0.3),        
        keras.layers.Dense(1, activation="sigmoid")
    ]
)
model.summary()

c:\Python\Python 3.11.9\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         7,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,777 (546.00 KB)

 Trainable params: 139,777 (546.00 KB)

 Non-trainable params: 0 (0.00 B)

## Train the model with `class_weight` argument

In [13]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2),
    loss="binary_crossentropy",
    metrics=metrics
)

In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint("model_checkpoints/fraud_model_at_epoch_{epoch}.keras")
]

class_weight = {0: weight_for_0, 1: weight_for_1}

<img src='https://drive.google.com/uc?export=view&id=1j8SxKYEi12jzJXi_bPO28q5SV9emuu0Y'>

In [15]:
model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)


Epoch 1/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - fn: 25.6549 - fp: 20099.8223 - loss: 3.3321e-06 - precision: 0.0080 - recall: 0.8640 - val_fn: 9.0000 - val_fp: 1294.0000 - val_loss: 0.0820 - val_precision: 0.0485 - val_recall: 0.8800
Epoch 2/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - fn: 18.2212 - fp: 4389.2568 - loss: 1.2563e-06 - precision: 0.0470 - recall: 0.9280 - val_fn: 10.0000 - val_fp: 328.0000 - val_loss: 0.0356 - val_precision: 0.1654 - val_recall: 0.8667
Epoch 3/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - fn: 11.9469 - fp: 2902.4336 - loss: 1.0767e-06 - precision: 0.0753 - recall: 0.9452 - val_fn: 6.0000 - val_fp: 2121.0000 - val_loss: 0.1218 - val_precision: 0.0315 - val_recall: 0.9200
Epoch 4/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - fn: 11.3363 - fp: 4031.2654 - loss: 9.3553e-07 - precision: 0.0509 - recall: 0.9567 - val_fn: 9.0000 - val_fp: 754.0000 - val_loss: 0.0901 - val_precision: 0.0805 - val_recall: 0.8800
Epoch 5/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 

#**Дополнительные материалы**


**По теории:**
1. [Interactive neural network playground in your browser](http://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.72732&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)
2. Backprop in depth by cs231 - https://cs231n.github.io/optimization-2/
3. Минусы и плюсы функций активаций (видео) - https://youtu.be/Gs8T_qF-FAA
4. A recipe for training neural networks  - http://karpathy.github.io/2019/04/25/recipe/
5. Метод обратного распространения ошибки (видео) - https://youtu.be/EuhoXsuu8SQ
6. Примерно аналогичный материал с лекции, только от Стенфорда и на английском - http://cs231n.github.io/neural-networks-1/#nn
7. Official intro to Keras - https://keras.io/getting_started/intro_to_keras_for_researchers/
8. Deep learning frameworks (видео) - https://www.youtube.com/watch?v=ghZyptkanB0


**По практике:**
1. Метод fit в Keras (видео) - https://youtu.be/PLlic60dgS4
2. Callbacks в Keras. ModelCheckpoint, ReduceLROnPlateau, EarlyStopping (видео) - https://youtu.be/sq9HvLAsK94